In [6]:
!unzip "/content/COVID.zip" -d "/content/"

Archive:  /content/COVID.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/COVID.zip or
        /content/COVID.zip.zip, and cannot find /content/COVID.zip.ZIP, period.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
type = ["COVID", "NON_COVID"]

os.chdir('/content/data')
if os.path.isdir('train/COVID/') is False: 
    os.mkdir('train')
    os.mkdir('valid')
    

    for i in type:
        shutil.move(f'{i}', 'train')
        os.mkdir(f'valid/{i}')

        valid_samples = random.sample(os.listdir(f'train/{i}'), 50)
        for j in valid_samples:
            shutil.move(f'train/{i}/{j}', f'valid/{i}')
os.chdir('../..')

FileNotFoundError: ignored

In [ ]:
train_path = '/content/data/train'
valid_path = '/content/data/valid'


train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(240,240), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(240,240), batch_size=10)


In [ ]:
img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.1),
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomTranslation(height_factor=(-0.1, 0.1), width_factor=(-0.3, 0.3)),
        preprocessing.RandomZoom(height_factor=(-0.3,0.3))
    ],
    name="img_augmentation",
)

In [ ]:
inputs = layers.Input(shape=(240, 240, 3))
x = img_augmentation(inputs)

In [ ]:
efficient = tf.keras.applications.EfficientNetB1(input_tensor=x, weights='imagenet', include_top=False)

In [ ]:
efficient.summary()

In [ ]:
x = efficient.output

In [ ]:
x = GlobalAveragePooling2D()(x)

In [ ]:
x = BatchNormalization()(x)

In [ ]:
x = Dropout(0.2)(x)

In [ ]:
predictions = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(inputs=efficient.input, outputs=predictions)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=30)

In [ ]:
model.save("/content/modelli")